In [1]:
from comet_ml import Experiment

experiment = Experiment(api_key="oda8KKpxlDgWmJG5KsYrrhmIV", project_name="consensusnet")

COMET WARNING: We detected that you are running inside a Ipython/Jupyter notebook environment but we cannot save your notebook source code. Please be sure to have installed comet_ml as a notebook server extension by running:
jupyter comet_ml enable 
For more details, please refer to: https://www.comet.ml/docs/python-sdk/warnings-errors
COMET WARNING: Comet.ml support for Ipython Notebook is limited at the moment, automatic monitoring and stdout capturing is deactivated 
For more details, please refer to: https://www.comet.ml/docs/python-sdk/warnings-errors
COMET INFO: Experiment is live on comet.ml https://www.comet.ml/ajuric/consensusnet/c19ac46c435c4fb3a62c507cb741dda2



In [2]:
import numpy as np
from keras.models import Model
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization, Input, Dropout
from keras.layers import Conv1D, MaxPooling1D, Conv2D, MaxPool2D
from keras.callbacks import LearningRateScheduler, EarlyStopping, TensorBoard
from keras.regularizers import l2

/home/ajuric/.local/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## Load data

In [3]:
X_validate, y_validate = np.load('./dataset-n20-X-validate.npy'), np.load('./dataset-n20-y-validate.npy')
print(X_validate.shape)
print(y_validate.shape)

(3109078, 41, 5)
(3109078, 6)


## Model definition

In [ ]:
def lr_schedule(epoch, lr):
    if epoch > 50:
        if epoch % 10 == 0:
            return lr * 0.95
    return lr

lr_callback = LearningRateScheduler(lr_schedule)
callbacks = [lr_callback, 
             EarlyStopping(monitor='loss', patience=3),]
#              TensorBoard(log_dir=tensorboard_output_dir, write_images=True, write_grads=True, histogram_freq=5, batch_size=10000)]

input_shape = X_validate.shape[1:]
num_output_classes = y_validate.shape[1]

input_layer = Input(shape=input_shape)

conv_1 = Conv1D(filters=16, kernel_size=4, padding='same', activation='selu', kernel_regularizer=l2(1e-3))(input_layer)
pool_1 = MaxPooling1D(pool_size=(5), strides=1)(conv_1)

conv_2 = Conv1D(filters=32, kernel_size=4, padding='same', activation='selu', kernel_regularizer=l2(1e-3))(pool_1)
pool_2 = MaxPooling1D(pool_size=(4), strides=1)(conv_2)

conv_3 = Conv1D(filters=48, kernel_size=4, padding='same', activation='selu', kernel_regularizer=l2(1e-3))(pool_2)
pool_3 = MaxPooling1D(pool_size=(3), strides=1)(conv_3)

flatten = Flatten()(pool_3)

dn_1 = Dense(336, activation='selu')(flatten)
drop = Dropout(0.5)(dn_1)

predictions = Dense(num_output_classes, activation='softmax')(drop)

model = Model(input_layer, predictions)

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.summary()

batch_size = 10000
epochs = 150